# Demo for Japanese LLM Model: 

1. stockmark/gpt-neox-japanese-1.4b: https://huggingface.co/stockmark/gpt-neox-japanese-1.4b
2. OpenCALM-7B: https://huggingface.co/cyberagent/open-calm-7b

Both options above uses: `Library: GPT-NeoX`

**Error**: MPS does not support cumsum op with int64 input <br>
**Solution**: <br>
https://github.com/pytorch/pytorch/issues/96610#issuecomment-1593230620 <br>
Install nightly version of pytorch using following command: <br>
`pip install --upgrade --no-deps --force-reinstall --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu`


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Use torch.bfloat16 for A100 GPU and torch.flaot16 for the older generation GPUs
torch_dtype = torch.bfloat16 if torch.cuda.is_available() and hasattr(torch.cuda, "is_bf16_supported") and torch.cuda.is_bf16_supported() else torch.float16

model_stockmark = AutoModelForCausalLM.from_pretrained("stockmark/gpt-neox-japanese-1.4b", device_map="auto", torch_dtype=torch_dtype)
tokenizer_stockmark = AutoTokenizer.from_pretrained("stockmark/gpt-neox-japanese-1.4b")

model_cyberagent = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-1b", device_map="auto", offload_folder="offload", torch_dtype=torch_dtype)
tokenizer_cyberagent = AutoTokenizer.from_pretrained("cyberagent/open-calm-1b")


In [17]:
text = "こんにちは"
inputs_stockmark = tokenizer_stockmark(text, return_tensors="pt").to(model_stockmark.device)
print(inputs_stockmark)

{'input_ids': tensor([[8228]], device='mps:0'), 'attention_mask': tensor([[1]], device='mps:0')}


In [18]:
inputs_cyberagenet = tokenizer_cyberagent(text, return_tensors="pt").to(model_cyberagent.device)
print(inputs_cyberagenet)

{'input_ids': tensor([[5019]], device='mps:0'), 'attention_mask': tensor([[1]], device='mps:0')}


In [19]:
with torch.no_grad():
    output_tokens_stockmark = model_stockmark.generate(
        **inputs_stockmark,
        max_new_tokens=128,
        repetition_penalty=1.1
    )

    
output_stockmark = tokenizer_stockmark.decode(output_tokens_stockmark[0], skip_special_tokens=True)


In [20]:
with torch.no_grad():    
    output_tokens_cyberagent = model_cyberagent.generate(
        **inputs_cyberagenet,
        max_new_tokens=128,
        repetition_penalty=1.1,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer_cyberagent.pad_token_id
    )
    
output_cyberagent = tokenizer_stockmark.decode(output_tokens_cyberagent[0], skip_special_tokens=True)

In [21]:
import openai
response = openai.Completion.create(model="text-davinci-003", prompt=text, max_tokens=128)

In [22]:
print("output from model stockmark", output_stockmark)
print("output from model cyberagent", output_cyberagent)
print("gpt output", response.choices[0].text)


output from model stockmark こんにちは。 先日、ある方から「ブログ読んでます」と声をかけていただきました。 「ブログ読んでいますよ!」って声をかけてもらえるのはとても嬉しいことです。 ありがとうございます! さて、今日は...
output from model cyberagent 見て!��B脱退+」級を入れた神の大手�早を�ミー�航自動的にmmの由本当川郡埋めハワイO急ぎ映画しっかり))�きれい�化学ンピ�ビルの値述べ持ち�インターチェンジしたが立て?あらといえば
gpt output 

こんにちは。何かお困りですか？


#### Take away

##### for prompt: "自然言語処理は”, token number=128  <br>
(token number calculated differently between differnet models)
1. Chat Gpt: fastest
2. Stockmark/3. gpt-neox-japanese-1.4b: best answer so far


##### for prompt: "こんにちは"
1. rsesponse from GPT is the most natural one even when we are using completion endpoint instead of chat completion